# Milestone Project 1: Food Vision Big

See the annotated version (plenty of text and explanations for each part) of this notebook on GitHub: [https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb)

And see the full course materials (including exercises and extra-curriculum) on GitHub: [https://github.com/mrdbourke/tensorflow-deep-learning/](https://github.com/mrdbourke/tensorflow-deep-learning/)

Video N°206: Making sure we have access to the right GPU for mixed precision training